
# FACT TABLE

In [0]:
    df = spark.sql("select * from aws_db_retail_metastore.silver.orders_silver")

    df.limit(10).display()

In [0]:
# Fact tables should only have numeric columns
# so we need to work on it

In [0]:
df_dim_cust = spark.sql("select dim_cust_id, customer_id from aws_db_retail_metastore.gold.dim_customers")

df_dim_prod = spark.sql("select product_id as dim_product_id, product_id from aws_db_retail_metastore.gold.dim_products")

In [0]:
df_fact = df.join(df_dim_cust, on=df["customer_id"] ==  df_dim_cust["customer_id"], how='left').join(df_dim_prod, on=df["product_id"] == df_dim_prod["dim_product_id"], how='left')

df_fact.select("order_id","dim_cust_id","dim_product_id","order_date","quantity").display()


In [0]:
df_fact_new = df_fact.drop("dim_customer_id","customer_id","product_id")

df_fact_new.display()


upserting on fact table

In [0]:
from delta.tables import DeltaTable


if spark.catalog.tableExists("aws_db_retail_metastore.gold.fact_orders"):
    dlt_obj = DeltaTable.forName(spark, "aws_db_retail_metastore.gold.fact_orders")
    dlt_obj.alias("trg")\
        .merge(df_fact_new.alias("src"),"trg.order_id = src.order_id and trg.dim_cust_id = src.dim_cust_id and trg.dim_product_id = src.dim_product_id")\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()
else:
    df_fact_new.write.format("delta").option("path","s3://argha-associatede-s3/gold/fact_orders").saveAsTable("aws_db_retail_metastore.gold.fact_orders")

In [0]:
%sql

select * from aws_db_retail_metastore.gold.fact_orders;